In [ ]:
import os
import json
from openai import AzureOpenAI

    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

In [ ]:
assistant_id = "asst_bLvSlz40u6JfYR30LpRDqLTe"

assistant = client.beta.assistants.retrieve(assistant_id)
print(assistant)

In [ ]:
thread = client.beta.threads.create()
print(thread.id)

In [4]:
# Add a user question to the thread
msg = "Can you help me with a patient chart?"

#msg = "email me a summary of this conversation to ddiaz@live.com thread_id: thread_4i0QVGUqIbnjd5SmNgXOCHib"

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=msg
)

thread_messages = client.beta.threads.messages.list(thread.id)

In [5]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [ ]:
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

In [ ]:
from api.functions.email import send_summary

#send_summary("ddiaz@live.com", thread.id)

In [ ]:
from api.functions.patient import get_patient_information
from api.functions.patient import get_patient_chart
from api.functions.email import send_report
from api.functions.patient import review_chart

import time

max_steps = 10 
sleep = .5

#add the available functions here   
available_functions = {"get_patient_information": get_patient_information, "get_patient_chart": get_patient_chart, "send_summary": send_summary,"review_chart":review_chart}

try:
    cnt = 0
    while cnt < max_steps:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)        
        cnt += 1
        if run.status == "requires_action":
            tool_responses = []
            if (
                run.required_action.type == "submit_tool_outputs"
                and run.required_action.submit_tool_outputs.tool_calls is not None
            ):
                tool_calls = run.required_action.submit_tool_outputs.tool_calls

                for call in tool_calls:
                    if call.type == "function":
                        if call.function.name not in available_functions:
                            raise Exception("Function requested by the model does not exist")
                        function_to_call = available_functions[call.function.name]
                        print(call.function.arguments)
                        print(call.function.name)
                        tool_response = function_to_call(**json.loads(call.function.arguments))
                        tool_responses.append({"tool_call_id": call.id, "output": tool_response})

            run = client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread.id, run_id=run.id, tool_outputs=tool_responses
            )

            print(run.status)

        if run.status == "failed":
            print("Run failed.")
            break
        if run.status == "completed":
            break
        time.sleep(sleep)

except Exception as e:
    print(e)



In [2]:
import sys
import os

# Add the parent directory of 'api' to the sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Now you can import the review_chart function
from api.functions.reviewer import review_chart
review_chart("Thank you for the additional information. Here is a summary of the details you've provided for the patient chart:**Dispatch Info:**- Dispatched to a home for a 78-year-old male with chest pains.- Patient found sitting on a couch on the first floor with his wife and son present.**Chief Complaint:**- Chest pain**History:**- No prior history of chest pains- Diabetic- Seasonal allergies- Taking OTC medication for heartburn and body aches as needed- Last oral intake at 6 PM; EMS arrival at 7:30 PM**Assessment:**- Patient alert to person, place, time, and event- Vitals: HR 98, BP 130/84, Temp: 98.8, Resp rate: 14, Blood oxygen: 99%- No other symptoms reported- Mild discomfort, but nothing remarkable")



NameError: name 'review_chart2' is not defined

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))
print(run.status)